**Задание 1**
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

*   если источник traffic_source равен Yandex или Google, то в source_type ставится organic;
*   для источников paid и email из России ставим ad

*   для источников paid и email не из России ставим other
*   все остальные варианты берём из traffic_source без изменений


**МЕТОД LOC**

In [113]:
%%time
import pandas as pd

visit_log = pd.read_csv('visit_log.csv', sep=';', encoding='utf-8')

visit_log.loc[(visit_log['traffic_source'] == 'yandex') | (visit_log['traffic_source'] == 'google'), ['source_type']] = 'organic'
visit_log.loc[((visit_log['traffic_source'] == 'paid') | (visit_log['traffic_source'] == 'email')) & ( visit_log['region'] != 'Russia'), ['source_type']] = 'other'
visit_log.loc[((visit_log['traffic_source'] == 'paid') | (visit_log['traffic_source'] == 'email')) & (visit_log['region'] == 'Russia'), ['source_type']] = 'ad'

visit_log.head(10)
# для проверки
#visit_log.loc[(visit_log['region'] != 'Russia') & (visit_log['traffic_source'] == 'email'), ['region', 'traffic_source', 'source_type']]


CPU times: user 90.3 ms, sys: 6.27 ms, total: 96.5 ms
Wall time: 216 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


In [ ]:
visit_log_pivot = visit_log.pivot_table(index='region', columns='source_type', values='traffic_source', aggfunc='count')
visit_log_pivot.head(10)

**Через функцию**

In [115]:
%%time
import pandas as pd

visit_log = pd.read_csv('visit_log.csv', sep=';', encoding='utf-8')

def source_type(row):
  if row['region'] == 'Russia' and  (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email'):
    return 'ad'
  elif row['region'] != 'Russia' and (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email'):
    return 'other'
  elif row['traffic_source'] == 'yandex' or  row['traffic_source'] == 'google':
    return 'organic'

visit_log['source_type'] = visit_log.apply(source_type, axis=1)

visit_log.head(10)
#visit_log_pivot = visit_log.pivot_table(index='region', columns='source_type', values='traffic_source', aggfunc='count')
#visit_log_pivot.head(10)

CPU times: user 319 ms, sys: 9.5 ms, total: 329 ms
Wall time: 328 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,None
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,None
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,None
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


**Задание 2**
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:


1.   Прочитайте содержимого файла с датафрейм.
2.   Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.



In [118]:
from operator import index
import pandas as pd
URLs = pd.read_csv('URLs.txt', sep=',')
URLs.head()


URLs[URLs['url'].str.contains('/[0-9]{8}-', case=False)].head()
#URLs[URLs['url'].str.contains('/[0-9]{8}-', regex=True)]  #  В чём принципиальная разница между case=False и regex=True

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


**Задание 3**
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [121]:
import pandas as pd
ratings = pd.read_csv('/content/ratings.csv', sep=',')
ratings.head(3)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [122]:
#  1_Вариант создания СВОДНОЙ ТАБЛИЦЫ
ratings_pivot = ratings.groupby(['userId']).agg(MIN_TIME=('timestamp','min'), MAX_TIME=('timestamp','max'), COUNT=('rating', 'count'))
ratings_pivot.sort_values(by='COUNT', ascending=False).head()

,MIN_TIME,MAX_TIME,COUNT
userId,,,
414,961436216,1527978072,2698
599,1498456035,1519558046,2478
474,974666330,1238350606,2108
448,1019123866,1499620709,1864
274,1171172369,1318410436,1346


In [124]:
#  2_Вариант создания СВОДНОЙ ТАБЛИЦЫ
ratings_pivot = ratings.groupby(['userId']).agg({'timestamp': ['min', 'max', 'count']})['timestamp'].sort_values(by='count', ascending=False)
ratings_pivot.head()


,min,max,count
userId,,,
414,961436216,1527978072,2698
599,1498456035,1519558046,2478
474,974666330,1238350606,2108
448,1019123866,1499620709,1864
274,1171172369,1318410436,1346


In [125]:
%%time
#  Вариант с функцией
def T_Life (row_table):
  T_ = row_table['max'] - row_table['min']
  if row_table['count'] > 100:
    return T_

ratings_pivot['T_Life'] = ratings_pivot.apply(T_Life, axis=1)
ratings_pivot[ratings_pivot['count'] > 100]

ratings_pivot['T_Life'].mean()

CPU times: user 15.2 ms, sys: 0 ns, total: 15.2 ms
Wall time: 22.2 ms


44731367.73469388

In [126]:
%%time
#  Вариант с функцией loc
ratings_pivot.loc[ratings_pivot['count'] > 100, ['T_Life']] = (ratings_pivot['max'] - ratings_pivot['min'])

mean_ = ratings_pivot['T_Life'].mean()
print(f'Cреднее время жизни пользователей, которые выставили более 100 оценок - {mean_}')

Cреднее время жизни пользователей, которые выставили более 100 оценок - 44731367.73469388
CPU times: user 3.14 ms, sys: 1.07 ms, total: 4.21 ms
Wall time: 4 ms


**Задание 4.** Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:


1.   таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
2.   аналогичную таблицу по типам выручки с указанием адреса клиента.



**К домашнему заданию №4**
Дана статистика услуг перевозок клиентов компании по типам:

*   rzd - железнодорожные перевозки
*   auto - автомобильные перевозки
*   air - воздушные перевозки
*   client_base - адреса клиентов

In [76]:
import pandas as pd

In [77]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [78]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [79]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [80]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [85]:
# РЕШЕНИЕ
concat_tabl_revenue = pd.concat([rzd, auto, air])
concat_tabl_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,NaN
3,114,5774.0,NaN,NaN
4,115,981.0,NaN,NaN
0,113,NaN,57483.0,NaN
1,114,NaN,83.0,NaN
2,115,NaN,912.0,NaN
3,116,NaN,4834.0,NaN
4,117,NaN,98.0,NaN


In [129]:
#таблица с тремя типами выручки для каждого client_id без указания адреса клиента

pivot_tabl_revenue = concat_tabl_revenue.groupby('client_id').sum()
#pivot_tabl_revenue['SUMMA'] = pivot_tabl_revenue.loc[:,['rzd_revenue', 'auto_revenue', 'air_revenue']].sum(axis=1)
pivot_tabl_revenue.eval('SUMMA = rzd_revenue + auto_revenue + air_revenue', inplace=True)
pivot_tabl_revenue.sort_values(by='SUMMA', ascending=False, inplace=True)
pivot_tabl_revenue

#Для проверки суммы и потери данных (84622.0)
#pivot_tabl_revenue['SUMMA'].sum()

84622.0

In [130]:
# Аналогичная таблица по типам выручки с указанием адреса клиента
pivot_tabl_revenue.merge(client_base, how='left', on='client_id')

#Для проверки суммы и потери данных (84622.0)
#pivot_tabl_revenue['SUMMA'].sum()

,client_id,rzd_revenue,auto_revenue,air_revenue,SUMMA,address
0,113,10283.0,57483.0,0.0,67766.0,Левобережная 1а
1,114,5774.0,83.0,0.0,5857.0,Мира 14
2,116,0.0,4834.0,4.0,4838.0,Строителей 18
3,112,2810.0,0.0,0.0,2810.0,Энтузиастов 8а
4,115,981.0,912.0,81.0,1974.0,ЗЖБИиДК 1
5,111,1093.0,0.0,0.0,1093.0,Комсомольская 4
6,118,0.0,0.0,173.0,173.0,Мастеркова 4
7,117,0.0,98.0,13.0,111.0,Панфиловская 33
